In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'module')))
from openTable import *
from filepath import *
from preprocessing import preprocessing_text as pre

import warnings
warnings.filterwarnings('ignore')

from re import sub
import pandas as pd

# import spacy
from spacy.lang.id import Indonesian,stop_words
nlp = Indonesian()  # use directly
stopwords = stop_words.STOP_WORDS 
stopwords |= {"nya","jurusan","the","of"}

def preprocessing(text):
    text = pre.remove_tag(text) #Remove Tag
    text = pre.remove_whitespace(text) #Remove Whitespace
    text = pre.lower(text) #Lower
    text = pre.remove_link(text) #Remove Link
    text = pre.alphabet_only(text) #Get Alphabet
    text = sub(r'sobat pintar','',text) # sorry:(
    text = pre.remove_whitespace(text) #Remove Whitespace
    text = [token.text for token in nlp(text)] #Token
    text = pre.slang(text)
    text = [token.lemma_ for token in nlp(text) if token.lemma_ not in stopwords] #Lemma & stopword
    
    return " ".join(text.strip())



def get_data():
    data = open_table(['entryId','content'],'BlogsEntry')
    
    return data

def cleaning():
    #get data
    data = get_data()
    data = data.head()
    data = rename_column(data,{0:'entryId', 1:'content'})
    data['clean'] = data.content.apply(preprocessing) #Preprocessing
    
    return data

In [4]:
data = cleaning()
display(data.head())
print(len(data.entryId.unique()))

,entryId,content,clean
0,204420,<p>Biasanya kakak di blog www.Herniawan.com me...,"[kakak, blog, bahas, tips, kena, bahas, stan, ..."
1,288433,<p>Kamu merasa tak punya peluang untuk ikut SN...,"[luang, snmptn, nilai, rapormu, turun, ombak, ..."
2,288455,<p>Emang gak tertarik sama SNMPTN (hmm... akib...,"[tarik, snmptn, hmm, akibat, nilai, rapor, flu..."
3,288478,"<p>Saat mendengar kata belajar, apa yang muncu...","[dengar, ajar, muncul, dikepalamu, kesal, mala..."
4,291921,"<p>Kamu masuk kelas sesuai jadwal, pulang seko...","[masuk, kelas, suai, jadwal, pulang, sekolah, ..."


767


In [15]:
#Save
data.to_csv('../data/data_berita_clean.csv', sep=',',header=data.columns.tolist(),index=None)

In [5]:
#Fetch wordcount for each content
def for_content(text):
    text = pre.remove_tag(text) #Remove Tag
    text = pre.remove_whitespace(text) #Remove Whitespace
    text = pre.lower(text) #Lower
    text = pre.remove_link(text) #Remove Link
    text = pre.alphabet_only(text) #Get Alphabet
    text = sub(r'sobat pintar','',text) # sorry:(
    text = pre.remove_whitespace(text) #Remove Whitespace
    
    return text
data.content = data.content.apply(for_content)
data['word_count'] = data.content.apply(lambda x: len(str(x).split(" ")))
data.head()

,entryId,content,clean,word_count
0,204420,biasanya kakak di blog membahas masalah serta ...,"[kakak, blog, bahas, tips, kena, bahas, stan, ...",3058
1,288433,kamu merasa tak punya peluang untuk ikut snmpt...,"[luang, snmptn, nilai, rapormu, turun, ombak, ...",323
2,288455,emang gak tertarik sama snmptn hmm akibat dari...,"[tarik, snmptn, hmm, akibat, nilai, rapor, flu...",426
3,288478,saat mendengar kata belajar apa yang muncul di...,"[dengar, ajar, muncul, dikepalamu, kesal, mala...",346
4,291921,kamu masuk kelas sesuai jadwal pulang sekolah ...,"[masuk, kelas, suai, jadwal, pulang, sekolah, ...",366


In [6]:
##Descriptive statistics of word counts
data.word_count.describe()

count     767.000000
mean      553.026076
std       143.020300
min       290.000000
25%       471.000000
50%       551.000000
75%       628.000000
max      3058.000000
Name: word_count, dtype: float64

In [11]:
#Identify common words
freq = pd.Series(' '.join(data.content).split()).value_counts()[:20]
freq

yang       12451
dan         9466
kamu        4968
dengan      4812
di          4413
pada        4281
ini         3868
atau        3763
untuk       3520
kita        2985
jurusan     2974
bisa        2754
tak         2687
juga        2586
dari        2496
dalam       2358
lebih       2239
lain        2087
akan        2025
menjadi     1956
dtype: int64

In [12]:
# Check stopword
set(freq.index.tolist()) - stopwords

{'jurusan'}

In [14]:
#Identify uncommon words
freq = pd.Series(' '.join(data.content).split()).value_counts()[-20:]
freq

ngisengin         1
gulick            1
gosipan           1
atuh              1
fih               1
suffer            1
santapan          1
daunnya           1
sepotong          1
bernoda           1
talentscouting    1
menggencet        1
alokasikan        1
edutopia          1
tahapannya        1
ts                1
plat              1
stengaah          1
memanaskan        1
tersendat         1
dtype: int64